In [53]:
import numpy as np
import pandas as pd
import os

In [54]:
os.path.exists('value_analysis.xlsx')

True

# 初看数据

In [16]:
try:
    isinstance(df, pd.DataFrame)
except:
    df = pd.read_excel('rfm_data.xlsx')
df.head()

,id,orderID,userID,goodsID,orderAmount,payment,chanelID,platfromType,orderTime,payTime,chargeback
0,1,sys-2018-254118088,user-157213,PR000064,272.51,272.51,渠道-0396,APP,2018-02-14 12:20:36,2019-02-28 13:38:41,否
1,2,sys-2018-263312190,user-191121,PR000583,337.93,337.93,渠道-0765,Wech atMP,2018-08-14 09:40:34,2019-01-01 14:47:14,是
2,3,sys-2018-188208169,user-211918,PR000082,905.68,891.23,渠道-0530,We c hatMP,2018-11-02 20:17:25,2019-01-19 20:06:35,否
3,4,sys-2018-203314910,user-201322,PR000302,786.27,688.88,渠道-0530,WEB,2018-11-19 10:36:39,2019-08-07 12:24:35,否
4,5,sys-2018-283989279,user-120872,PR000290,550.77,542.51,渠道-9527,APP,2018-12-26 11:19:16,2019-10-01 07:42:43,否


In [17]:
df_2019 = df.query('orderTime.dt.year == 2019')

In [18]:
# 存放数据统计量
data_count = pd.Series({})

C:\Users\11912\AppData\Local\Temp/ipykernel_5752/461063351.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data_count = pd.Series({})


In [19]:
data_count['shape'] = df_2019.shape

In [20]:
df_2019.goodsID.value_counts()

PR000895    354
PR000762    350
PR000103    335
PR000587    303
PR000385    302
           ... 
PR000347     21
PR000597     19
PR000468     18
PR000986     16
PR001000     13
Name: goodsID, Length: 1001, dtype: int64

In [21]:
df_2019.info()
# chanelID null
# object -> float

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104296 entries, 5 to 104300
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            104296 non-null  int64         
 1   orderID       104296 non-null  object        
 2   userID        104296 non-null  object        
 3   goodsID       104296 non-null  object        
 4   orderAmount   104296 non-null  float64       
 5   payment       104296 non-null  float64       
 6   chanelID      104288 non-null  object        
 7   platfromType  104296 non-null  object        
 8   orderTime     104296 non-null  datetime64[ns]
 9   payTime       104296 non-null  datetime64[ns]
 10  chargeback    104296 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 9.5+ MB


In [22]:
data_count['user_num'] = df_2019.userID.unique().size
data_count

shape       (104296, 11)
user_num           79211
dtype: object

# 数据清洗

In [23]:
df.head(1)

,id,orderID,userID,goodsID,orderAmount,payment,chanelID,platfromType,orderTime,payTime,chargeback
0,1,sys-2018-254118088,user-157213,PR000064,272.51,272.51,渠道-0396,APP,2018-02-14 12:20:36,2019-02-28 13:38:41,否


## 异常情况

In [24]:
# id
df_2019 = df_2019.query('id >= 0')

# 支付时间早于下单时间。订单金额小于0。
df_2019 = df_2019[df_2019['payTime'] > df_2019['orderTime']]
df_2019 = df_2019.query('orderAmount >= 0')
df_2019 = df_2019.query('payment >= 0')

# 下单时间与付款时间太长 24h
df_2019 = df_2019[(df_2019['payTime'] - df_2019['orderTime']).dt.days < 1]

# 去除重复
df_2019 = df_2019.drop_duplicates(subset=['orderID'])

# drop space
df_2019['platfromType'] = df_2019['platfromType'].str.replace(' ', '')

In [25]:
# drop null
df_2019.dropna(inplace=True)

In [26]:
data_count['shape']  = df_2019.shape

In [27]:
df_2019.value_counts('platfromType')

platfromType
APP           52127
WechatMP      42702
WEB            7012
ALiMP          1984
WechatShop       87
Wap               3
dtype: int64

# 各种指标

In [28]:
# 交易总金额（GMV）
total_order = df_2019['orderAmount'].sum()
total_pay = df_2019['payment'].sum()

real_order = df_2019.query('chargeback =="否"')['orderAmount'].sum()
real_pay = df_2019.query('chargeback =="否"')['payment'].sum()

# 每个用户平均消费
avg_pay = real_pay / df_2019['userID'].nunique()

avg_order = df_2019['orderAmount'].count() / df_2019['userID'].nunique()

In [29]:
data_count['total_order_amount'] =  total_order
data_count['total_pay'] =  total_pay
data_count['real_order'] =  real_order
data_count['real_pay'] =  real_pay
data_count['avg_pay'] =  avg_pay
data_count['avg_order'] =  avg_order
data_count['goodsNum'] = df_2019['goodsID'].nunique()

In [30]:
data_count

shape                     (103915, 11)
user_num                         79211
total_order_amount        109159624.37
total_pay             121453096.139917
real_order                 94605788.25
real_pay              105456122.999844
avg_pay                    1335.107334
avg_order                     1.315596
goodsNum                          1001
dtype: object

## 退单率

In [31]:
chargeback_count = df_2019[df_2019['chargeback'] == '是'].shape[0]
chargeback_rate = chargeback_count / df_2019.shape[0]
data_count['chargeback_rate'] = chargeback_rate
chargeback_rate

0.13179040561997787

In [32]:
def back_rate(fea:str):
    df = df_2019.query('chargeback =="是"').groupby(f'{fea}', as_index=False).size()
    add = df_2019.groupby(f'{fea}', as_index=False).size()['size']
    assert add.count() == df.shape[0]
    df['total'] = add
    df['rate'] = df['size']/df['total']
    return df

In [33]:
goods_back = back_rate('goodsID')
goods_back.sort_values('rate', ascending=True)

,goodsID,size,total,rate
440,PR000440,1,60,0.016667
210,PR000210,2,75,0.026667
966,PR000966,1,37,0.027027
136,PR000136,2,62,0.032258
478,PR000478,1,30,0.033333
...,...,...,...,...
958,PR000958,10,35,0.285714
1000,PR001000,4,13,0.307692
147,PR000147,21,68,0.308824
676,PR000676,14,45,0.311111



    淘宝：淘宝的退货率在3%~10%左右。
    京东：京东的退货率相对比较低，大致在1%~5%之间。
    天猫：天猫的退货率大约在1%~10%之间。
    苏宁易购：苏宁易购的退货率大概在1%~10%之间。
    唯品会：唯品会的退货率相对较高，一般在10%~20%之间。


一般来说，如果退货率在10%以下，说明卖家的产品质量和服务都比较好，买家比较满意；如果退货率在10%到20%之间，说明卖家的产品质量和服务存在一定的问题，需要加以改进；如果退货率在20%以上，说明卖家的产品质量和服务存在比较严重的问题，需要尽快解决。

## 用户复购率

复购率/回购率：重复购买客户数量客户样本数量

In [34]:
order_count_by_user = df_2019.groupby(['userID'])['orderID'].count()
order_count_by_user.head(3)

userID
user-100000    1
user-100003    1
user-100006    2
Name: orderID, dtype: int64

In [35]:
reorder = order_count_by_user[order_count_by_user >= 2]
repurchase_rate = reorder.size / order_count_by_user.size
data_count['repurchase_rate'] = repurchase_rate
repurchase_rate

0.25682707280944966

一般来说，电商平台的复购率在5%到20%之间比较常见，该平台的复购率为25.29%，复购率较高。说明用户粘性较高

## 用户生命周期

In [36]:
life_time = (df_2019.groupby(['userID'])['orderTime'].max() - df_2019.groupby(['userID'])['orderTime'].min()).dt.days
gcut = pd.cut(life_time[life_time>0],
              10,
              right=False		#右闭为否
             )
gcut_count = gcut.value_counts(sort=False)
life_time[life_time>0].mean()

128.0562243027691

In [37]:
from pyecharts.charts import Line, Bar
from pyecharts import options as opts

bar = (
    Bar(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis(gcut_count.index.map(lambda x:x.__str__()).to_list())
    .add_yaxis('下单量', gcut_count.to_list())
)

bar.render_notebook()

+ 人均用户的平均生命周期挺长的
+ 在200天左右可以适当刺激用户消费，引导持续消费

## 时间段分析

### 每月

In [38]:
df_2019['order_month'] = df_2019['orderTime'].dt.month

In [39]:
month_gmv = df_2019.groupby(by='order_month')['orderAmount'].sum()
month_gmv.head(3)

order_month
1    7008563.76
2    5558607.33
3    6865662.15
Name: orderAmount, dtype: float64

In [40]:
from pyecharts.charts import Line, Bar
from pyecharts import options as opts

line = (
    Line(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis([f'{i}月' for i in month_gmv.index.tolist()])
    .add_yaxis('GMV', month_gmv.values.tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    ]
            ))
    .set_global_opts(yaxis_opts=opts.AxisOpts(name='GMV',is_scale=True),
                            xaxis_opts=opts.AxisOpts(name='Month'))
)

line.render_notebook()


+ 2月GMV最低，可能是由于大家都线下采购年货导致的，并且由于春节是法定节假日，物流和电商也都放假，GMV自然不会高。
+ 第一季度的GMV总体偏低，从第二季度4~5月份开始大幅上升。
+ 5~8月GMV较高，
+ 9~10月份出现小幅度下跌，可能是由于学生开学导致的。
+ 11~12月份平台GMV情况良好，可能是由于双11和双12两个电商购物节导致的。


In [41]:
# 分平台
def channel_group(ch:str):
    if ch == 'wechat':
        s = 'platfromType == "WechatMP" | platfromType == "Wechatshop"'
    elif ch == 'app':
        s = 'platfromType == "APP"'
    else:
        s = 'platfromType != "APP" and platfromType != "WechatMP" and platfromType != "Wechatshop"'
    
    data = df_2019.query(s).groupby(by='order_month')['orderAmount'].sum()
    return data

wechat = channel_group('wechat')
app = channel_group('app')
el = channel_group('else')

bar = (
    Bar(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis([f'{i}月' for i in month_gmv.index.tolist()])
    .add_yaxis('Wechat', wechat.values.tolist())
    .add_yaxis('APP', app.values.tolist())
    .add_yaxis('Else', el.values.tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    ]
            ))
    .set_global_opts(yaxis_opts=opts.AxisOpts(name='GMV',
                                              is_scale=True),
                     xaxis_opts=opts.AxisOpts(name='Month'),
                     datazoom_opts= [opts.DataZoomOpts(range_start=10, range_end=80,is_zoom_lock=False)]
                     )
)

bar.render_notebook()

In [42]:
wechat_rate = wechat / wechat.iloc[0]
app_rate = app / app.iloc[0]
el_rate = el / el.iloc[0]

line = (
    Line(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis([f'{i}月' for i in month_gmv.index.tolist()])
    .add_yaxis('Wechat', wechat_rate.values.tolist())
    .add_yaxis('APP', app_rate.values.tolist())
    .add_yaxis('Else', el_rate.values.tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    ]
            ),
                     #markline_opts=opts.MarkLineOpts(
                     #    data = [opts.MarkLineItem(type_="average", name="均值")]
                     #)
                     )
    .set_global_opts(yaxis_opts=opts.AxisOpts(name='GMV',
                                              is_scale=True),
                     xaxis_opts=opts.AxisOpts(name='Month'),
                     datazoom_opts= [opts.DataZoomOpts(range_start=10, range_end=80,is_zoom_lock=False)]
                     )
)

line.render_notebook()

由于不同渠道之间差异较大，关心其增长率，以一月为基准
+ 所有平台在二月的交易是最少的，春节时期用户更倾向于线下等快速获得货物的方式
+ 5-9 月，微信与app保持平稳，而其他平台，如网页端有较大的增幅
+ 6 月 与 11,12月是微信与app交易量最高的两个峰值，分别对应年中与年底促销的活动

### 时段

In [43]:
order_hour = df_2019['orderTime'].dt.hour
def time_segment(hour):
    if 0 <= hour < 6:
        return '凌晨'
    elif 6 <= hour < 9:
        return '早晨'
    elif 9 <= hour < 11:
        return '上午'
    elif 11 <= hour < 14:
        return '中午'
    elif 14 <= hour < 17:
        return '下午'
    elif 17 <= hour < 19:
        return '傍晚'
    else:
        return '晚上'
df_2019['order_hour'] = order_hour
df_2019['order_time_seg'] = order_hour.map(time_segment)

In [44]:
time_seg_count = df_2019.groupby('order_time_seg')['id'].count().sort_values(ascending=False)
x = time_seg_count.index.tolist()
y = time_seg_count.values.tolist()

bar = (
    Bar(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis(x)
    .add_yaxis('下单量', y)
)

bar.render_notebook()

事实上，12-14 是最多的

但是晚上这个时段设的很长

## 分平台分析

# RFM用户分层



+ RFM模型主要包括以下三个指标：
  + R：Recency，最近一次购买时间。
  + F: Frequency，购买的频次。
  + M: Monetary，购买总金额。

+ 接着，对计算结果进行评分，根据三个指标的评分可以将用户分为八大类。


In [45]:
# 以2020年1月1日为时间点
import datetime

end_day = datetime.datetime(2020, 1, 1)
# .days 计算差值，非日期数据 timedelta64[ns]
# .day  日期数据 datetime64[ns]
df_2019['r_day'] = (end_day - df_2019['orderTime']).dt.days

In [46]:
rfm_mode_data = df_2019.groupby('userID').agg({
    'r_day': 'min',         # 最近购买
    'orderID': 'count',     # 次数
    'payment': 'sum'        # 总金额
}).rename(
    columns={
        'r_day': 'R-Score',
        'orderID': 'F-Score',
        'payment': 'M-Score'
    }
)
rfm_mode_data.head()

,R-Score,F-Score,M-Score
userID,,,
user-100000,79,1,1770.81
user-100003,221,1,511.59
user-100006,47,2,2771.98
user-100007,351,1,2162.14
user-100008,45,1,4879.94


In [47]:
r_ref = rfm_mode_data['R-Score'].mean()
f_ref = rfm_mode_data['F-Score'].mean()
m_ref = rfm_mode_data['M-Score'].mean()

rfm_mode_data['R'] = (rfm_mode_data['R-Score'] < r_ref).astype(np.int32)
rfm_mode_data['F'] = (rfm_mode_data['F-Score'] > f_ref).astype(np.int32)
rfm_mode_data['M'] = (rfm_mode_data['M-Score'] > m_ref).astype(np.int32)

In [48]:
rfm = rfm_mode_data.loc[:, ['R', 'F', 'M']]
def trans_tag(x):
    tag_list = {
        '111': '重要价值用户',
        '101': '重要发展用户',
        '011': '重要保持用户',
        '001': '重要挽留用户',
        '110': '一般价值用户',
        '100': '一般发展用户',
        '010': '一般保持用户',
        '000': '一般挽留用户'
    }
    
    key = ''.join([str(i) for i in x.values])
    return tag_list.get(key, 'RFM评分错误')

In [49]:
rfm['tag'] = rfm.apply(trans_tag, axis=1)
rfm = rfm.reset_index()
rfm_count = rfm['tag'].value_counts()

In [51]:
if not os.path.exists('value_analysis.xlsx'):
    rfm.to_excel('value_analysis.xlsx')

In [122]:
rfm_count.head(3)

一般挽留用户    24918
一般发展用户    21735
重要价值用户     8878
Name: tag, dtype: int64

In [198]:
from pyecharts.charts import Pie
from pyecharts import options as opts

data = [tuple(z) for z in zip(rfm_count.index.tolist(), rfm_count.values.tolist())]

def plot_pie(title, label, data, rose = False):
    if rose:
        rose = 'radius'
    pie = (
        Pie(init_opts=opts.InitOpts(theme = 'dark'))
        # radius 设置内径和外径
        .add(label, data, radius=['40%', '80%'], rosetype= rose)
        .set_global_opts(
            legend_opts=opts.LegendOpts(
                type_="scroll", 
                pos_left="75%", 
                pos_top="32.5%",
                orient="vertical"
            ),
            title_opts=opts.TitleOpts(
                title=title, 
                pos_left='center', 
                pos_top='center',
                title_textstyle_opts=opts.TextStyleOpts(
                    color='#00BFFF', 
                    font_size=28, 
                    font_weight='bold'
                ),
            )
        )
        .set_series_opts(label_opts=opts.LabelOpts(
            # {b|{b}: } b| 指后面的设置， {b} 指代表的内容
            formatter="\n{b|{b}: }{d|{d}%}\n",
            rich={
                # 设置 b，d
                    "b": {"color": "#00c6d7","fontSize": 16, "lineHeight": 33, "fontWeight":700},
                    "d": {"color": "#fff000","fontSize": 16, "lineHeight": 33, "fontWeight":900},
                },),
        )
    )
    return pie
pie = plot_pie('用户类型','用户等级', data)

pie.render_notebook()

+ 重要保持用户数量，重要挽留用户较少，占22.8%。用户粘性相对较低
+ 高价值用户数量较少。重要价值用户吸引和维护高价值客户方面存在一些问题。
+ 发展用户潜力。一般发展用户数量相对较多。

1. 重要价值用户：VIP服务。
  + 用户特性：距离上次消费时间近，消费频次和金额都较高。
  + 运营策略：重点维护，由专人组成VIP客户服务团队，给予专享权益、定期福利等活动，让其体验到与众不同的感受，在精神和物质上都给予激励，让他们主动分享和转介绍。
2. 重点发展用户：想办法提高消费频率。
  + 用户特性：距离上次消费时间较近，消费金额高，但消费频次不高。
  + 运营策略：这类用户忠诚度不高，需要让用户在平台上活跃起来，不断优化产品和服务，制定相应策略，让用户提升使用和购买频率。
3. 重点保留用户：主动联系。
  + 用户特性：距离上次消费时间较远，但是消费频次和金额都较高。
  + 运营策略：定期进行邮件推送、软件内部推送、短信推送等，主动与客户保持联系和互动。
4. 重点挽留用户：分析哪里处理问题。
  + 用户特性：距离上次消费时间较远，消费频次不高，但上一次消费的金额较高。
  + 运营策略：重点防止流失，提前做好流失预警，配合相应策略触达行为。
5. **一般价值用户**：
  + 用户特性：距离上次消费时间较近，消费频次高，但是消费金额低。
  + 运营策略：这类用户是重点潜力用户，可以发放不同形式的优惠券，引导不断增加消费或者投入。
6. 一般发展用户：
  + 用户特性：距离上次消费时间较近，消费频次低，消费金额也低。
  + 运营策略：这类用户可能属于**新用户**，需加强其对回复的工作，并提供新手福利，并在各种节假日活动时候及时提醒。
7. 一般保留用户：
  + 用户特性：距离上次消费时间较远，消费频次高，消费金额低。
  + 运营策略：这类用户是预流失用户，前期曾经活跃过，但后期发现产品、服务、奖励等无法满足预期需求，就不再进行复购行为了，运营者需要对其做好利益和情感的触达。
8. 一般挽留用户：
  + 用户特性：距离上次消费时间较远，消费频次低，消费金额低。
  + 运营策略：这部分用户可能已流失一段时间，不易挽回，在预算有限的情况下，暂且考虑放弃。 



## 不同类型用户对平台的意义

In [124]:
df_2019 = pd.merge(df_2019, rfm[['userID', 'tag']], on = 'userID')

In [125]:
group_value = df_2019.groupby('tag')[['orderAmount']].sum().sort_values('orderAmount', ascending=False)
# 每用户贡献的流水
group_value['per_order'] = df_2019.groupby('tag')[['orderAmount']].sum() / df_2019.groupby('tag')[['orderAmount']].count()

In [126]:
data = [tuple(z) for z in zip(group_value.index.tolist(), group_value['orderAmount'].tolist())]
pie = plot_pie('', '', data)
pie.render_notebook()

In [127]:
group_value = group_value.sort_values('per_order', ascending=False)

data = [tuple(z) for z in zip(group_value.index.tolist(), group_value['per_order'].tolist())]
pie = plot_pie('', '', data)
pie.render_notebook()

+ 重要用户占三成，但是在总交易金额与人均交易金额都占了 **近七成** 
+ 一般发展用户和一般挽留用户基数较大，也带来较大的交易额

### 分析一下折扣率

In [128]:
df_2019['discount'] = df_2019['payment'] / df_2019['orderAmount']

In [129]:
discount = df_2019.query('discount<=1').groupby('tag')['discount'].mean()
discount

tag
一般价值用户    0.888808
一般保持用户    0.899819
一般发展用户    0.898679
一般挽留用户    0.909709
重要价值用户    0.937295
重要保持用户    0.944658
重要发展用户    0.969358
重要挽留用户    0.974484
Name: discount, dtype: float64

In [130]:
df_2019.query('discount<=1').groupby('order_month')['discount'].mean()

order_month
1     0.938982
2     0.941597
3     0.941069
4     0.937750
5     0.901257
6     0.921898
7     0.909329
8     0.909855
9     0.896708
10    0.900600
11    0.921089
12    0.931575
Name: discount, dtype: float64

In [131]:
df_2019

,id,orderID,userID,goodsID,orderAmount,payment,chanelID,platfromType,orderTime,payTime,chargeback,order_month,order_hour,order_time_seg,r_day,tag,discount
0,6,sys-2019-279103297,user-146548,PR000564,425.20,425.20,渠道-0765,WechatMP,2019-01-01 00:12:23,2019-01-01 00:13:37,否,1,0,凌晨,364,一般挽留用户,1.000000
1,7,sys-2019-316686066,user-104210,PR000709,1764.37,1707.04,渠道-0396,WechatMP,2019-01-01 00:23:06,2019-01-01 00:23:32,否,1,0,凌晨,364,重要价值用户,0.967507
2,80204,sys-2019-323014107,user-104210,PR000743,532.33,227.63,渠道-0765,WechatMP,2019-10-24 12:08:27,2019-10-24 12:08:53,否,10,12,中午,68,重要价值用户,0.427611
3,8,sys-2019-306447069,user-104863,PR000499,499.41,480.42,渠道-0007,WechatMP,2019-01-01 01:05:50,2019-01-01 01:06:17,否,1,1,凌晨,364,重要保持用户,0.961975
4,9661,sys-2019-416338629,user-104863,PR000760,3244.83,3159.35,渠道-9527,WechatMP,2019-02-19 19:43:54,2019-02-19 19:44:09,否,2,19,晚上,315,重要保持用户,0.973657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103910,104297,sys-2019-344079195,user-182248,PR000813,831.29,766.07,渠道-0896,WechatMP,2019-12-31 23:32:55,2019-12-31 23:33:06,否,12,23,晚上,0,一般发展用户,0.921544
103911,104298,sys-2019-296195955,user-143322,PR000938,1565.67,1414.89,渠道-0007,APP,2019-12-31 23:33:05,2019-12-31 23:34:36,是,12,23,晚上,0,一般发展用户,0.903696
103912,104299,sys-2019-382387725,user-220484,PR000782,3326.83,3273.54,渠道-0530,WEB,2019-12-31 23:37:30,2019-12-31 23:37:44,否,12,23,晚上,0,重要发展用户,0.983982
103913,104300,sys-2019-303891464,user-285133,PR000096,241.75,241.75,渠道-0765,WechatMP,2019-12-31 23:38:43,2019-12-31 23:39:01,是,12,23,晚上,0,一般发展用户,1.000000


In [132]:
bar = (
    Bar(init_opts=opts.InitOpts(theme = 'dark'))
    .add_xaxis(discount.index.str.slice(0,4).tolist())
    .add_yaxis('Discount', discount.values.tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="最大值"),
                    opts.MarkPointItem(type_="min", name="最小值"),
                    ]
            ))
    .set_global_opts(yaxis_opts=opts.AxisOpts(name='GMV',
                                              is_scale=True),
                     xaxis_opts=opts.AxisOpts(name='Month'),
                     datazoom_opts= [opts.DataZoomOpts(range_start=10, range_end=80,is_zoom_lock=False)]
                     )
)

bar.render_notebook()

+ 对一般价值用户的折扣率较低，运营方针合适
+ 重要发展用户折扣率较高，后续运营过程可以适当降低，吸引用户


## 商品分析

In [ ]:
df_2019['goodsID'].value_counts()

PR000895    351
PR000762    349
PR000103    335
PR000587    302
PR000060    301
           ... 
PR000347     21
PR000597     19
PR000468     18
PR000986     16
PR001000     13
Name: goodsID, Length: 1001, dtype: int64

In [144]:
good_data = df_2019.query('chargeback =="否"').groupby('goodsID', as_index=False).agg({'orderAmount': 'sum', 'orderID':'count'}).sort_values('orderAmount', ascending=False)
good_data = good_data.rename(
    columns={
        'orderID': 'order_num'
    }
)
good_data['avg_price'] = good_data['orderAmount'] / good_data['order_num']
good_data['back_rate'] = goods_back['rate']

In [199]:
data = good_data.iloc[:10]
data = [tuple(z) for z in zip(data.goodsID.tolist(), data.orderAmount.tolist())]
pie = plot_pie('商品交易比例','商品', data, 1)

pie.render_notebook()

In [146]:
good_data

,goodsID,orderAmount,order_num,avg_price,back_rate
385,PR000385,450388.57,268,1680.554366,0.106667
61,PR000061,374880.84,229,1637.034236,0.125954
103,PR000103,369670.55,297,1244.681987,0.113433
405,PR000405,347211.09,161,2156.590621,0.134409
345,PR000345,332089.79,260,1277.268423,0.106529
...,...,...,...,...,...
408,PR000408,22073.47,37,596.580270,0.075000
964,PR000964,15936.32,34,468.715294,0.128205
986,PR000986,14270.97,12,1189.247500,0.250000
1000,PR001000,10530.10,9,1170.011111,0.307692


In [147]:
th = 0.2
good_data.query(f'back_rate>{th}')['orderAmount'].sum(), good_data.query(f'back_rate>{th}')['goodsID'].count()

(1423465.3, 31)

退单率超0.2的商品共31件，实际交易金额 1420000，这些商品质量不太高，需要加强管理

In [148]:
th = 0.1
good_data.query(f'back_rate<{th}')['orderAmount'].sum(), good_data.query(f'back_rate<{th}')['goodsID'].count()

(14740999.759999998, 178)

In [162]:
imp_good_data = df_2019[df_2019.tag.str.startswith('重要')].query('chargeback =="否"').groupby('goodsID', as_index=False).agg({'orderAmount': 'sum', 'orderID':'count'}).sort_values('orderAmount', ascending=False)
imp_good_data = imp_good_data.rename(
    columns={
        'orderID': 'order_num'
    }
)
imp_good_data['avg_price'] = imp_good_data['orderAmount'] / imp_good_data['order_num']
imp_good_data['back_rate'] = goods_back['rate']

In [163]:
imp_good_data

,goodsID,orderAmount,order_num,avg_price,back_rate
385,PR000385,385954.33,155,2490.027935,0.106667
405,PR000405,311462.22,122,2552.969016,0.134409
61,PR000061,303524.75,129,2352.905039,0.125954
720,PR000720,296567.60,127,2335.177953,0.125604
345,PR000345,256864.76,113,2273.139469,0.106529
...,...,...,...,...,...
855,PR000855,7820.84,6,1303.473333,0.186047
408,PR000408,7382.52,4,1845.630000,0.075000
347,PR000347,5351.85,7,764.550000,0.238095
1000,PR001000,3442.62,3,1147.540000,0.307692


In [164]:
data = imp_good_data.iloc[:10]
data = [tuple(z) for z in zip(data.goodsID.tolist(), data.orderAmount.tolist())]

pie = plot_pie('商品交易比例','商品', data)

pie.render_notebook()

与所用用户的前十差不多